# Conformal Geometric Algebra

##  Intro

Conformal Geometric Algebra (CGA) is a projective geometry tool which allows conformal transformations to be implemented with rotations.  To do this,  the original geometric algebra is extended by two dimensions, one of positive signature $e_+$ and one of negative signature $e_-$. It is convenient to define a *null* basis given by 

$$
e_o =  \frac{1}{2}(e_- -e_+)\\
e_\infty = e_-+e_+
$$

A vector in the original space $x$ is *up-projected* into a conformal vector $X$  by 

$$X = x + \frac{1}{2} x^2 e_{\infty} +e_o $$


To map a conformal vector back into a vector from the original space, the vector is first normalized, then rejected from the minkowski plane $E_0$,


$$ X = \frac{X}{X \cdot e_{\infty}}$$

then 

$$x = X \wedge E_0\, E_0^{-1}$$


To implement this in `clifford`  we could create your own CGA by instantiating the CGA, like `Cl(3,1)`  for example, and then making the definitions and maps described above. 
Or, we you can use the helper function `conformalize()`. 

## using `conformalize()`
The purpose of `conformalize` is to remove the redunancy assocaited with creating a conformal geometric algebras. `conformalize` takes an existing geometric algebra layout and *conformalizes* it by adding two dimensions, as described above. Additionally, this function returns a new layout for the CGA, a dict of blades for the CGA, and dictionary containing the added basis vectors and up/down projection functions. 

To demonstrate we will conformalize $G_2$, producing a CGA of $G_{3,1}$.

In [54]:
from clifford import Cl, conformalize

G2, blades_g2 = Cl(2)

blades_g2 # inspect the blades


{'e1': (1.0^e1), 'e12': (1.0^e12), 'e2': (1.0^e2)}

Now, conformalize it 

In [55]:
G2c, blades_g2c, stuff = conformalize(G2)

blades_g2c   #inspect the CGA blades



{'e1': (1.0^e1),
 'e12': (1.0^e12),
 'e123': (1.0^e123),
 'e1234': (1.0^e1234),
 'e124': (1.0^e124),
 'e13': (1.0^e13),
 'e134': (1.0^e134),
 'e14': (1.0^e14),
 'e2': (1.0^e2),
 'e23': (1.0^e23),
 'e234': (1.0^e234),
 'e24': (1.0^e24),
 'e3': (1.0^e3),
 'e34': (1.0^e34),
 'e4': (1.0^e4)}

Additionally lets inspect `stuff` 

In [56]:
stuff

{'E0': (1.0^e34),
 'down': <function clifford.<lambda>>,
 'einf': (1.0^e3) + (1.0^e4),
 'en': (1.0^e4),
 'eo': -(0.5^e3) + (0.5^e4),
 'ep': (1.0^e3),
 'homo': <function clifford.<lambda>>,
 'up': <function clifford.<lambda>>}

It contains the following:
* `ep` - postive basis vector added
* `en` - negative basis vector added
* `eo` - zero vecror of null basis (=.5*(en-ep))
* `einf` - infinity vector of null basis (=en+ep)
* `E0` - minkowski bivector (=einf^eo)
* `up()` - function to up-project a vector from GA to CGA
* `down()` - function to down-project a vector from CGA to GA
* `homo()` - function ot homogenize a CGA vector


we can put the blades and the stuff into the local namespace, 

In [57]:
locals().update(blades_g2c)
locals().update(stuff)

As a sanity check, make sure down/up are consistent

In [58]:

down(up(e1+e2)) == e1+e2

True

## Operations

Conformal transformations in $G_n$ are achieved through versers in the conformal space $G_{n+1,1}$. These versers can be categorized by their relation to the added minkowski plane, $E_0$. There are three categories,

* verser purely in $E_0$
* verser partly in $E_0$
* verser out of $E_0$


A visual model for conformal space with the relavant subspaces labeled is shown below. 

In [59]:
from IPython.display import Image
Image(url='_static/conformal space.svg')

### Versers purely in $E_0$

First we generate some vectors in G2, which we can operate on

In [60]:
a= 1*e1 + 2*e2
b= 3*e1 + 4*e2

#### Inversions

$ e_+ X e_+$

Inversion is a reflection in hyperplane normal to $e_-$, this swaps $e_o$  and $e_{\infty}$

In [61]:
assert(down(ep*up(a)*ep)  == a.inv())

#### Involutions 

$E_0 X E_0$

In [62]:
assert(down(E0*up(a)*E0) == -a)

#### Dialations

$D_\alpha  =  e^{-\frac{\ln{\alpha}}{2} \,E_0} $

$D_\alpha \, X \, \tilde{D_\alpha} $

In [63]:
D = lambda alpha: e**((-log(alpha)/2.)*(E0)) 
alpha = rand()
assert(down( D(alpha)*up(a)*~D(alpha)) == (alpha*a))


### Versers partly in $E_0$

#### Translations

$$ V = e ^{\frac{1}{2} e_{\infty} a } = 1 + e_{\infty}a$$

In [64]:
T = lambda x: e**(1/2.*(einf*x)) 
assert(down( T(a)*up(b)*~T(a)) == b+a)

#### Transversions

A transversion is an inversion,  followed by a translation, followed by a inversion. The verser is 

$$V= e_+ T_a e_+$$

which is recognised as the translation bivector reflected in the $e_+$ vector. From the diagram, it is seen that this is equivalent to the bivector in $x\wedge e_o$,

$$ e_+ (1+e_{\infty}a)e_+ $$

$$ e_+^2 + e_+e_{\infty}a e_+$$
$$2 +2e_o a$$

the factor of 2 may be dropped, because the conformal vectors are null


In [65]:
V = ep * T(a) * ep
assert ( V == 1+(eo*a))

K = lambda x: 1+(eo*a) 

B= up(b)
assert( down(K(a)*B*~K(a)) == 1/(a+1/b) ) 

### Versers Out of  $E_0$

Versers that are out of $E_0$ are made up of the versers within the original space. These include reflections and rotations, and their conformal representation is identical to their form in $G^n$, except the minus sign is dropped for reflections, 

#### Reflections

$$ -mam^{-1}  \rightarrow MA\tilde{M}  $$

In [66]:
m = 5*e1 + 6*e2
n = 7*e1 + 8*e2


assert(down(m*up(a)*m) == -m*a*m.inv())


#### Rotations

$$ mnanm = Ra\tilde{R} \rightarrow RA\tilde{R}  $$

In [67]:
R = lambda theta: e**((-.5*theta)*(e12))
theta = pi/2
assert(down( R(theta)*up(a)*~R(theta)) == R(theta)*a*~R(theta))


## Combinations of Operations

### simple example

As a simple example consider the combination  operations of translation,scaling, and inversion. 

$$b=-2a+e_0 \quad \rightarrow \quad B= (T_{e_0}E_0 D_2) A \tilde{ (D_2 E_0 T_{e_0})} $$

In [68]:
A = up(a)
V = T(e1)*E0*D(2)
B = V*A*~V
assert(down(B) == (-2*a)+e1 )

### Transversion

A transversion may be built from a inversion, translation, and inversion. 

$$c = (a^{-1}+b)^{-1}$$

In conformal GA, this is accomplished by 

$$C = VA\tilde{V}$$

$$V= e_+ T_b e_+$$

In [69]:
A = up(a)
V = ep*T(b)*ep
C = V*A*~V
assert(down(C) ==1/(1/a +b))


### Rotation about a point 

Rotation about a point, $a$  can be achieved by translating the origina to $a$ then rotating, then translating back.  Just like the transversion can be thought of as translating the involution operator, rotation about a point can also be thought of as  translating the Rotor itself. Covariance.  